In [30]:
# load packages
import os
import time
import random
import urllib.parse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [31]:
# set working directory
os.chdir("C:/Users/eafra/Documents/The Shepherd Films")

In [32]:
# set search terms
search_terms = "car detailing atlanta"

In [33]:
# set driver options
options = Options()
options.add_argument("--start-maximized")

# specify path to chromedriver
service = Service("C:/Users/eafra/Documents/drivers/chromedriver.exe")

# load driver using Service object
driver = webdriver.Chrome(service=service, options=options)

# load webpage
driver.get(f"https://www.google.com/maps/search/{urllib.parse.quote(search_terms)}/")

# wait for webpage to load and locate side container
results_pane = driver.find_element(By.CSS_SELECTOR, f"div[aria-label='Results for {search_terms}']")

# scroll to make all results appear
keep_scrolling = True

while(keep_scrolling):
    results_pane.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    html = driver.find_element(By.TAG_NAME, "html").get_attribute("outerHTML")
    if(html.find("You've reached the end of the list.") != -1):
        keep_scrolling = False

# click the back to the top button
back_to_top_button = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/button')

back_to_top_button.click()

# get all profile elements
profiles = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "hfpxzc"))
)

# get number of profiles
num_profiles = len(profiles)

# create empty list
rows = []

# get business names, urls, and descriptions in a loop
for i in range(num_profiles):

    # get business name
    name = profiles[i].get_attribute('aria-label')

    # scroll to profile and click on it
    action = ActionChains(driver)
    action.move_to_element(profiles[i]).click().perform()
    
    # get url
    time.sleep(random.uniform(2, 5))
    try:
        url_element = driver.find_element(By.CSS_SELECTOR, "a[class='CsEnBe'][data-tooltip='Open website']")
        url = url_element.get_attribute('href')
        
    except NoSuchElementException:
        url = "no website found"
    
    # get description
    time.sleep(random.uniform(2, 5))
    description = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='DkEaL ']"))
        ).text
    
    # collect data
    rows.append({"name": name, "url": url, "description": description})

# create data frame once all rows are collected
df = pd.DataFrame(rows)

# close browser session
driver.quit()

In [34]:
# export to csv file
df.to_csv(search_terms + ".csv", index = False)